In [1]:
import os
import numpy as np
import pandas as pd

### Step - 1
 - First extract dataframe of title.akas.tsv file. 
 - Collect title ids of those records whose language or region is unknown or is not popular (number of occurrences of those languages less than 30K or non-top-8 langugaes).
 - Drop columns of ordering and attributes as they doesn't seems to be important features.
 - Store all these removable or irrelevant titleIds in a Set named `irrelevant_titleIds` and this removes around `17%` of the data from titles.akas.tsv file

In [4]:
title_akas_df = pd.read_csv('../data/latest-tsvs/title.akas.tsv', sep= '\t')
title_akas_df.head()

/Users/abhinivesh/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [19]:
len( title_akas_df[ (title_akas_df.language == '\\N') | (title_akas_df.region == '\\N') ] )

4118884

In [39]:
language_counts = title_akas_df.language.value_counts()

In [49]:
top_languages = set( language_counts[ (language_counts > 30000) & (language_counts.index != '\\N') ].index )

In [52]:
languages_to_be_removed = set( language_counts[ (language_counts < 30000) | (language_counts.index == '\\N') ].index )

In [56]:
len( title_akas_df.loc[ title_akas_df.language.isin(languages_to_be_removed) ] )

4250720

In [59]:
title_akas_df = title_akas_df.drop( columns= ['ordering', 'attributes' ] )

In [60]:
title_akas_df.head()

,titleId,title,region,language,types,isOriginalTitle
0,tt0000001,Карменсіта,UA,\N,imdbDisplay,0
1,tt0000001,Carmencita,DE,\N,\N,0
2,tt0000001,Carmencita - spanyol tánc,HU,\N,imdbDisplay,0
3,tt0000001,Καρμενσίτα,GR,\N,imdbDisplay,0
4,tt0000001,Карменсита,RU,\N,imdbDisplay,0


In [61]:
irrelevant_titleIds = set()

In [66]:
removable_akas_ids = set( title_akas_df.loc[ title_akas_df.language.isin(languages_to_be_removed) ].titleId.tolist() )

In [70]:
irrelevant_titleIds = irrelevant_titleIds.union(removable_akas_ids)

### Step - 2
 - Extract dataframe of title.basics.tsv
 - Remove records whose title doesn't have both start year and end year, (or) doesn't have any genres, (or) the titleType is not that significant like the last 4 values sorted in descending order of their occurrences.
 - Store all these removable or irrelevant titleIds in the same Set named `irrelevant_titleIds` and this removes around `41%` of the data from titles.akas.tsv file

In [72]:
title_basics_df = pd.read_csv('../data/latest-tsvs/title.basics.tsv', sep= '\t')
title_basics_df.head()

/Users/abhinivesh/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [74]:
title_basics_df.titleType.value_counts()

tvEpisode       5250669
short            776521
movie            557038
video            282535
tvSeries         195015
tvMovie          110980
tvMiniSeries      33977
tvSpecial         30087
videoGame         26586
tvShort           26219
Name: titleType, dtype: int64

In [79]:
title_types_value_counts = title_basics_df.titleType.value_counts()
removable_title_types = set( title_types_value_counts[ title_types_value_counts < 40000 ] )

In [88]:
removable_titleIds = title_basics_df.loc[ (title_basics_df.startYear == '\\N') & (title_basics_df.endYear == '\\N') | 
                (title_basics_df.genres == '\\N') | (title_basics_df.titleType.isin(removable_title_types) ) ].tconst.tolist()

In [90]:
removable_titleIds = set(removable_titleIds)

In [92]:
irrelevant_titleIds = irrelevant_titleIds.union(removable_titleIds)

In [94]:
len(irrelevant_titleIds), len( set( title_akas_df.titleId ) )

(3319808, 5125534)

In [96]:
len( title_akas_df.loc[ title_akas_df.titleId.isin(irrelevant_titleIds) ] ) / len(title_akas_df)

0.41190693123525424